## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input

from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

# 定義輸入張量
input_tensor = Input(shape=(32, 32, 3))

# Xception 架構
include_top = False  # 設定為 True 來包含 fully connected layer，False 來排除
xception_base = tf.keras.applications.Xception(include_top=include_top,
                                               weights='imagenet',
                                               input_tensor=input_tensor)

if include_top:
    model = xception_base
else:
    # 加入自定義的分類層
    x = xception_base.output
    x = GlobalAveragePooling2D()(x)  # 全域平均池化
    x = Dense(10, activation='softmax')(x)  # 假設你有 10 個分類

    # 建立模型
    model = Model(inputs=xception_base.input, outputs=x)

# 顯示模型摘要
model.summary()

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 32, 32, 3)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1 (Conv2D)     │ (None, 15, 15, 32)     │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1_bn           │ (None, 15, 15, 32)     │            128 │ block1_conv1[0][0]     │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1_act          │ (None, 15, 15, 32)     │              0 │ block1_conv1_bn[0][0]  │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2 (Conv2D)     │ (None, 13, 13, 64)     │         18,432 │ block1_conv1_act[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2_bn           │ (None, 13, 13, 64)     │            256 │ block1_conv2[0][0]     │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2_act          │ (None, 13, 13, 64)     │              0 │ block1_conv2_bn[0][0]  │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv1           │ (None, 13, 13, 128)    │          8,768 │ block1_conv2_act[0][0] │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv1_bn        │ (None, 13, 13, 128)    │            512 │ block2_sepconv1[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2_act       │ (None, 13, 13, 128)    │              0 │ block2_sepconv1_bn[0]… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2           │ (None, 13, 13, 128)    │         17,536 │ block2_sepconv2_act[0… │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2_bn        │ (None, 13, 13, 128)    │            512 │ block2_sepconv2[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 7, 7, 128)      │          8,192 │ block1_conv2_act[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_pool               │ (None, 7, 7, 128)      │              0 │ block2_sepconv2_bn[0]… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization  

 Total params: 20,881,970 (79.66 MB)

 Trainable params: 20,827,442 (79.45 MB)

 Non-trainable params: 54,528 (213.00 KB)

## 添加層數

In [7]:
input_tensor = Input(shape=(32, 32, 3))

# Xception 架構
model2 = tf.keras.applications.Xception(include_top=False,
                                               weights='imagenet',
                                               input_tensor=input_tensor, pooling = None, classes = 10)
x = model2.output

x = GlobalAveragePooling2D()(x)  # 全域平均池化
x = Dense(128, activation='relu')(x)
x = Dropout(0.1)(x)
predictions = Dense(10, activation='softmax')(x)  # 假設你有 10 個分類

model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))

Model深度： 137


## 鎖定特定幾層不要更新權重

In [8]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [9]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test


## Normalize Training and Testset
x_train, x_test = normalize(x_train, x_test)

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
(50000, 32, 32, 3)


In [11]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 10)
(10000, 10)


## Training

In [13]:
model = tf.keras.applications.Xception(include_top=False,
                                               weights='imagenet',
                                               input_tensor=input_tensor, pooling = None, classes = 10)
x = model.output

x = GlobalAveragePooling2D()(x)  # 全域平均池化
x = Dense(128, activation='relu')(x)
x = Dropout(0.1)(x)
predictions = Dense(10, activation='softmax')(x)  # 假設你有 10 個分類

model = Model(inputs=model.input, outputs=predictions)

In [14]:
!pip install tensorflow

import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model

# Assuming 'model' is your Keras Functional model
class MyModel(tf.keras.Model):  # Or your existing model class
    def __init__(self, model):
        self.model = model

    def call(self, inputs, training=None, mask=None):
        # Your model logic here
        x = self.dense1(inputs)
        return self.dense2(x)

# Re-instantiate your model using the modified class
model = MyModel()  # Or your existing model instantiation

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)

TypeError: MyModel.__init__() got an unexpected keyword argument 'inputs'

In [12]:
# Assuming 'model' is your Keras Functional model
class Model(tf.keras.Model):  # Or your existing model class
    def call(self, inputs, training=None, mask=None):
        # Your existing model logic here

        # Add debugging statements
        print("Input shape:", inputs.shape)
        for i, layer in enumerate(self.layers):
            inputs = layer(inputs)
            print(f"Layer {i} output shape:", inputs.shape)

        return inputs

# Re-instantiate your model using the modified class
model = Model()  # Or your existing model instantiation

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)

Epoch 1/100


NotImplementedError: Exception encountered when calling Model.call().

[1mModel Model does not have a `call()` method implemented.[0m

Arguments received by Model.call():
  • args=('tf.Tensor(shape=(None, 32, 32, 3), dtype=float32)',)
  • kwargs=<class 'inspect._empty'>

In [10]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)

Epoch 1/100


KeyError: 'Exception encountered when calling Functional.call().\n\n\x1b[1m140008835427456\x1b[0m\n\nArguments received by Functional.call():\n  • inputs=tf.Tensor(shape=(None, 32, 32, 3), dtype=float32)\n  • training=True\n  • mask=None'

In [17]:


# 假設 'x_train' 和 'y_train' 是你的訓練資料
# 輸入數據的shape為 (32, 32, 3)，假設有 10 個分類

input_tensor = tf.keras.Input(shape=(32, 32, 3))

# 定義自定義的 Keras 模型類
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        # 定義 Xception backbone
        self.base_model = Xception(include_top=False,
                                   weights='imagenet',
                                   input_tensor=input_tensor)
        # 定義自訂的全連接層
        self.global_pooling = GlobalAveragePooling2D()
        self.dense1 = Dense(128, activation='relu')
        self.dropout = Dropout(0.1)
        self.predictions = Dense(10, activation='softmax')  # 10 個分類

    def call(self, inputs, training=None, mask=None):
        # 前向傳播邏輯
        x = self.base_model(inputs, training=training)
        x = self.global_pooling(x)
        x = self.dense1(x)
        x = self.dropout(x, training=training)
        return self.predictions(x)

# 使用自定義類別來實例化模型
model = MyModel()

# 編譯模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 假設 x_train 和 y_train 是你的訓練資料
# 進行訓練
model.fit(x_train, y_train, batch_size=32, epochs=10)


Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2884s 2s/step - accuracy: 0.4258 - loss: 1.6098
Epoch 2/10
 813/1563 ━━━━━━━━━━━━━━━━━━━━ 22:38 2s/step - accuracy: 0.7421 - loss: 0.7971

KeyboardInterrupt: 